In [1]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import BertTokenizerFast

from src.data_utils import get_clean_text, TextDataset, collate_fn
from src.lstm_model import LSTMAutoCopleteText
from src.lstm_traint import train_model

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Сбор и подготовка данных

In [2]:

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
texts = get_clean_text("./data/raw_dataset.csv")
train_data, val_data = train_test_split(texts, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(val_data, test_size=0.5, random_state=42)

train_dataset = TextDataset(train_data, tokenizer)
val_dataset = TextDataset(val_data, tokenizer)
test_dataset = TextDataset(test_data, tokenizer)

In [3]:
train_loader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=256,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

Обучение модели

In [4]:
model = LSTMAutoCopleteText(vocab_size=tokenizer.vocab_size)
train_model(model, train_loader, val_loader, tokenizer)

100%|██████████| 4995/4995 [08:49<00:00,  9.44it/s]


Epoch 1 | Train Loss: 6.247 | Val Loss: 5.816 | Val Accuracy: 15.90% | ROUGE1: 0.0632 | ROUGE2: 0.00598


100%|██████████| 4995/4995 [08:52<00:00,  9.39it/s]


Epoch 2 | Train Loss: 5.682 | Val Loss: 5.600 | Val Accuracy: 17.28% | ROUGE1: 0.07 | ROUGE2: 0.01


100%|██████████| 4995/4995 [08:52<00:00,  9.39it/s]


...

Epoch 4 | Train Loss: 5.415 | Val Loss: 5.409 | Val Accuracy: 18.45% | ROUGE1: 0.0667 | ROUGE2: 0.0119


100%|██████████| 4995/4995 [08:51<00:00,  9.39it/s]


Epoch 5 | Train Loss: 5.343 | Val Loss: 5.351 | Val Accuracy: 18.82% | ROUGE1: 0.0664 | ROUGE2: 0.00949


100%|██████████| 4995/4995 [08:53<00:00,  9.37it/s]


Epoch 6 | Train Loss: 5.288 | Val Loss: 5.311 | Val Accuracy: 19.07% | ROUGE1: 0.0684 | ROUGE2: 0.00968


In [6]:
import os, torch

os.makedirs("checkpoints", exist_ok=True)

ckpt = {
    "model_state": model.state_dict(),
    "epoch": 7,
    "model_config": {
        "vocab_size": model.embedding.num_embeddings,
        "emb_dim": model.embedding.embedding_dim,
        "hidden_dim": model.rnn.hidden_size,
        "num_layers": model.rnn.num_layers,
        "pad_id": getattr(model, "pad_id", 0),
        "tie_weights": bool(model.fc.weight is model.embedding.weight),
        "dropout": (model.dropout.p if hasattr(model.dropout, "p") else 0.0),
    },
    "tokenizer_name": getattr(tokenizer, "name_or_path", None),
}
torch.save(ckpt, "checkpoints/lstm_checkpoint.pt")
print("Saved to checkpoints/lstm_checkpoint.pt")

Saved to checkpoints/lstm_checkpoint.pt
